# Initialisation des librairies et du fichier

In [ ]:
# FACULTATIF
# à utiliser dans google colab
# uploader un fichier local depuis l'ordinateur

#from google.colab import files
#uploaded = files.upload()

In [ ]:
# Importation des bibliothèques nécessaires à l’analyse
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.core.dtypes.common import is_numeric_dtype  # utile pour détecter les colonnes numériques

# Lecture du fichier CSV brut (période 2013–2023)
df = pd.read_csv("eco2mix-regional-cons-def.csv",
                 sep=';',                   # séparateur confirmé
                 na_values=['ND', '-'],     # gestion des valeurs manquantes
                 low_memory=False,          # éviter erreurs de type
                 encoding='utf-8')

# Options d’affichage dans le notebook
pd.set_option('display.float_format', lambda x: '%.4f' % x)  # afficher les floats avec 4 décimales
pd.set_option('display.max_columns', None)                   # afficher toutes les colonnes, même si nombreuses

# Vérification du résultat
df.head()

,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 30
0,93,Provence-Alpes-Côte d'Azur,Données définitives,2013-01-01,00:00,2012-12-31T23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84,Auvergne-Rhône-Alpes,Données définitives,2013-01-01,00:00,2012-12-31T23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,76,Occitanie,Données définitives,2013-01-01,00:00,2012-12-31T23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28,Normandie,Données définitives,2013-01-01,00:00,2012-12-31T23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44,Grand Est,Données définitives,2013-01-01,00:00,2012-12-31T23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Gestion des types de données

In [ ]:
# Conversion des colonnes temporelles au bon format

# On transforme la colonne 'Date' (texte comme "2013-01-01") en vrai objet date pour pouvoir trier, grouper, extraire l'année, etc.
df['Date'] = pd.to_datetime(df['Date'])

# La colonne 'Date - Heure' contient déjà une date et une heure combinées (ex : "2012-12-31T23:00:00+00:00"),
# donc on peut directement la convertir en format datetime
df['Date - Heure'] = pd.to_datetime(df['Date - Heure'])

# La colonne 'Heure' contient uniquement l’heure (ex : "00:00"), on la convertit en objet time
# On ajoute 'errors="coerce"' pour éviter les erreurs en cas de format inattendu
df['Heure'] = pd.to_datetime(df['Heure'], format='%H:%M', errors='coerce').dt.time

# On convertit le code INSEE en chaîne de caractères, car c’est un identifiant (et non un nombre à calculer)
df['Code INSEE région'] = df['Code INSEE région'].astype('str')

# On crée une nouvelle colonne indiquant le jour de la semaine (0 = lundi, 6 = dimanche), à partir de la colonne 'Date'
df['JourSemaine'] = df['Date'].dt.weekday

# Aperçu global du DataFrame : types de colonnes et valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121408 entries, 0 to 2121407
Data columns (total 33 columns):
 #   Column               Dtype              
---  ------               -----              
 0   Code INSEE région    object             
 1   Région               object             
 2   Nature               object             
 3   Date                 datetime64[ns]     
 4   Heure                object             
 5   Date - Heure         datetime64[ns, UTC]
 6   Consommation (MW)    float64            
 7   Thermique (MW)       float64            
 8   Nucléaire (MW)       float64            
 9   Eolien (MW)          float64            
 10  Solaire (MW)         float64            
 11  Hydraulique (MW)     float64            
 12  Pompage (MW)         float64            
 13  Bioénergies (MW)     float64            
 14  Ech. physiques (MW)  float64            
 15  Stockage batterie    float64            
 16  Déstockage batterie  float64            
 17  Eolien t

# Nettoyage des lignes et colonnes vides

In [ ]:
# Suppression de la colonne vide "Column 30" (générée à cause d’un point-virgule en trop dans le CSV d’origine)
if 'Column 30' in df.columns:
    df = df.drop(columns=['Column 30'])

# Suppression des 12 premières lignes du fichier qui sont vides : reset_index(drop=True)
# évite de garder un index qui commencerait à 12 après suppression des 12 premières lignes
# recrée un index propre et continu à partir de 0
df = df.drop(index=range(12)).reset_index(drop=True)

# Gestion des doublons

In [ ]:
# Vérifie le nombre de lignes en double dans le DataFrame
# Cela permet de savoir s’il y a des doublons exacts (mêmes valeurs sur toutes les colonnes)
nb_doublons = df.duplicated().sum()
print(str(nb_doublons) + " doublons détectés")

# Suppression des lignes dupliquées (si nb_doublons > 0)
# drop_duplicates() garde la première occurrence de chaque doublon
df_clean = df.drop_duplicates()

0 doublons détectés


In [ ]:
# Vérification : taille du DataFrame avant et après suppression des doublons
# Cela permet de confirmer que le nettoyage a bien été appliqué et de voir combien de lignes ont été supprimées

print("Taille avant suppression :", df.shape)
print("Taille après suppression :", df_clean.shape)

# Nous voyons que 142 lignes dupliquées ont été supprimées.
# On passe de 2 103 540 à 2 103 398 lignes, ce qui est cohérent avec le nombre de doublons détectés précédemment.

Taille avant suppression : (2121396, 32)
Taille après suppression : (2121396, 32)


# Gestion des valeurs manquantes

---



In [ ]:
# Analyse des valeurs manquantes et des zéros dans chaque colonne

# On calcule :
# le nombre de valeurs manquantes (NaN) par colonne
# le pourcentage de NaN par colonne
# le nombre de valeurs exactement égales à 0 par colonne

pd.DataFrame({
    "Nb valeurs manquantes" : df_clean.isna().sum(),
    "% valeurs manquantes" : df_clean.isna().mean() * 100,
    "Nb valeurs = 0"        : (df_clean == 0).sum()
})

# Interprétation :
# Les colonnes 'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre' et 'Eolien offshore' sont vides à 80 % et ne contiennent que des zéros : elles seront supprimées
# Les colonnes 'TCO' et 'TCH' contiennent des NaN uniquement avant 2020 : on remplacera ces NaN par 0 car l’absence de mesure avant 2020 signifie une absence de production

,Nb valeurs manquantes,% valeurs manquantes,Nb valeurs = 0
Code INSEE région,0,0.0000,0
Région,0,0.0000,0
Nature,0,0.0000,0
Date,0,0.0000,0
Heure,0,0.0000,0
Date - Heure,0,0.0000,0
Consommation (MW),0,0.0000,0
Thermique (MW),0,0.0000,165861
Nucléaire (MW),701275,33.0572,183984
Eolien (MW),96,0.0045,58065


In [ ]:
# Vérification des valeurs manquantes avant et après 2020

# Certaines colonnes (TCO, TCH) ont été ajoutées dans le fichier à partir de 2020
# Il est donc normal qu’elles soient entièrement vides avant cette date

# On découpe le DataFrame en deux parties
df_bef_2020 = df.loc[df['Date'].dt.year < 2020]
df_aft_2020 = df.loc[df['Date'].dt.year >= 2020]

# On compare les valeurs manquantes avant et après 2020
pd.DataFrame({
    "Nb NaN avant 2020" : df_bef_2020.isna().sum(),
    "% NaN avant 2020"  : df_bef_2020.isna().mean() * 100,
    "Nb NaN après 2020" : df_aft_2020.isna().sum(),
    "% NaN après 2020"  : df_aft_2020.isna().mean() * 100
})

# Interprétation :
# Les colonnes TCO et TCH sont entièrement vides avant 2020 (100 % de NaN)
# Cela confirme qu'elles ont été ajoutées dans le fichier à partir de cette date
# Après 2020, elles contiennent des données
# On pourra donc conserver ces colonnes mais remplacer les NaN par 0 avant 2020
# Certaines colonnes (ex : 'Stockage batterie', 'Eolien offshore') restent très vides même après 2020
# On envisagera de les supprimer car elles ne contiennent que très peu d’informations utiles

,Nb NaN avant 2020,% NaN avant 2020,Nb NaN après 2020,% NaN après 2020
Code INSEE région,0,0.0000,0,0.0000
Région,0,0.0000,0,0.0000
Nature,0,0.0000,0,0.0000
Date,0,0.0000,0,0.0000
Heure,0,0.0000,0,0.0000
Date - Heure,0,0.0000,0,0.0000
Consommation (MW),0,0.0000,0,0.0000
Thermique (MW),0,0.0000,0,0.0000
Nucléaire (MW),613435,41.6667,87840,13.5315
Eolien (MW),96,0.0065,0,0.0000


In [ ]:
# Traitement des NaN par interpolation temporelle

# On trie le DataFrame avant interpolation pour que l’ordre temporel soit respecté
df_clean = df_clean.sort_values(by=['Date', 'Région'])

# Interpolation linéaire : remplit les NaN entre deux valeurs connues
df_clean = df_clean.interpolate(method='linear')

# Vérifie s'il reste des NaN après interpolation
print("Total de NaN restants après interpolation :", df_clean.isna().sum().sum())

# Interprétation :
# L'interpolation linéaire a permis de combler une partie des valeurs manquantes, mais certaines colonnes n'ont pas été traitées car presque entièrement vide.
# Cela confirme qu’il faudra supprimer ou traiter manuellement les colonnes encore problématiques.

C:\Users\Laurie\AppData\Local\Temp\ipykernel_7804\2052725508.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_clean = df_clean.interpolate(method='linear')


Total de NaN restants après interpolation : 24399172


In [ ]:
# Colonnes non interpolables (NaN + 0 uniquement)

# Certaines colonnes ont toujours des valeurs vides ou égales à zéro :
# 'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre', 'Eolien offshore'

# On vérifie les valeurs présentes dans ces colonnes
print("Stockage batterie :", df_clean['Stockage batterie'].unique())
print("Déstockage batterie :", df_clean['Déstockage batterie'].unique())
print("Eolien terrestre :", df_clean['Eolien terrestre'].unique())
print("Eolien offshore :", df_clean['Eolien offshore'].unique())

# Ces colonnes ne contiennent que NaN ou des 0, on peut donc les supprimer
df_clean = df_clean.drop(columns=[
    'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre', 'Eolien offshore'
])

# Interprétation :
# Les 4 colonnes ciblées ne contiennent aucune valeur utile (juste des NaN ou des 0)
# Elles ne sont pas exploitables dans le cadre de cette analyse
# Les supprimer permet d’alléger le dataset et de faciliter les traitements suivants

Stockage batterie : [nan  0.]
Déstockage batterie : [nan  0.]
Eolien terrestre : [nan  0.]
Eolien offshore : [nan  0.]


In [ ]:
# Vérification des colonnes contenant encore des NaN

# On cherche les colonnes ayant encore des valeurs manquantes
missing = df_clean.isna().sum()
print("Colonnes avec NaN restants (non traités par interpolation) :")
print(missing[missing > 0].sort_values(ascending=False))

# Interprétation :
# Les colonnes TCO (%) et TCH (%) contiennent encore 100 % de NaN
# Cela confirme qu’elles n’ont pas été traitées par interpolation (car entièrement vides avant 2020)
# On sait que ces colonnes ont été ajoutées à partir de 2020
# Il est donc logique de remplacer les NaN restants par 0 (absence de mesure avant 2020)

Colonnes avec NaN restants (non traités par interpolation) :
TCO Thermique (%)      1472244
TCH Thermique (%)      1472244
TCO Nucléaire (%)      1472244
TCH Nucléaire (%)      1472244
TCO Eolien (%)         1472244
TCH Eolien (%)         1472244
TCO Solaire (%)        1472244
TCH Solaire (%)        1472244
TCO Hydraulique (%)    1472244
TCH Hydraulique (%)    1472244
TCO Bioénergies (%)    1472244
TCH Bioénergies (%)    1472244
dtype: int64


In [ ]:
# Remplacer les NaN restants dans les colonnes TCO et TCH par 0

# Ces colonnes sont entièrement vides avant 2020 : on considère que 0 = pas de production
colonnes_vides = [
    'TCO Nucléaire (%)', 'TCH Nucléaire (%)',
    'TCH Thermique (%)', 'TCO Thermique (%)',
    'TCO Eolien (%)',    'TCH Eolien (%)',
    'TCO Solaire (%)',   'TCH Solaire (%)',
    'TCO Hydraulique (%)','TCH Hydraulique (%)',
    'TCO Bioénergies (%)','TCH Bioénergies (%)'
]

# On remplace les NaN par 0 dans ces colonnes
for col in colonnes_vides:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(0)

# Vérification : il ne doit plus rester de NaN
print("NaN restants après remplissage final :", df_clean.isna().sum().sum())

# Interprétation :
# Les colonnes TCO et TCH étaient vides uniquement avant 2020, car elles ne sont apparues que cette année-là
# Remplacer les NaN par 0 signifie qu'il n'y a pas de production mesurée avant cette date
# La vérification montre que le dataset est maintenant propre, complet et prêt à être exploité

NaN restants après remplissage final : 0


# Gestion des valeurs aberrantes

## Pourcentages

In [ ]:
# Traitement des valeurs aberrantes dans les colonnes de pourcentage (TCO, TCH)

# Les colonnes TCO (%) représentent la couverture des besoins d'une région par une filière de production.
# On accepte que certaines valeurs soient > 100 % : ça indique qu'une région produit plus qu’elle ne consomme.
# On ne modifie donc pas les TCO.

# Les colonnes TCH (%) représentent le taux de charge d'une filière par rapport à sa capacité maximale.
# Ces valeurs doivent obligatoirement rester entre 0 et 100 %.
# On corrige les valeurs supérieures à 100 % en les remplaçant par 100.

# Liste des colonnes TCH concernées
col_TCH = [
    'TCH Nucléaire (%)', 'TCH Thermique (%)', 'TCH Eolien (%)',
    'TCH Hydraulique (%)', 'TCH Solaire (%)', 'TCH Bioénergies (%)'
]

# 1 : Vérifier le nombre de valeurs supérieures à 100 % (donc non cohérentes)
display((df_clean[col_TCH] > 100).sum())

# 2 : Remplacer les valeurs > 100 % par 100
# Remplacer les valeurs aberrantes supérieures à 100 par 100 permet de rester dans les limites physiques réalistes.
for col in col_TCH:
    df_clean[col] = df_clean[col].apply(lambda x: 100 if x > 100 else x)

# 3 : Vérification après correction
# Il doit s'afficher 0 partout
display((df_clean[col_TCH] > 100).sum())

# 4 : Vérifier qu’il n’existe pas de valeurs < 0 dans les colonnes TCO et TCH (c'est ce qu'on appelle des anomalies)
# On s'assure qu’il n’y a pas d’erreurs de saisie ou de conversion.
col_TCO = [
    'TCO Nucléaire (%)', 'TCO Thermique (%)', 'TCO Eolien (%)',
    'TCO Hydraulique (%)', 'TCO Solaire (%)', 'TCO Bioénergies (%)'
]
col_TCH_TCO = col_TCH + col_TCO

# Afficher les anomalies < 0 s'il y en a
display((df_clean[col_TCH_TCO] < 0).sum())

# Interprétation :
# Toutes les colonnes de pourcentage (TCO, TCH) contiennent désormais des valeurs comprises entre 0 et 100 %.
# Le jeu de données est donc maintenant cohérent.

TCH Nucléaire (%)      1330
TCH Thermique (%)         0
TCH Eolien (%)          695
TCH Hydraulique (%)      89
TCH Solaire (%)           1
TCH Bioénergies (%)      74
dtype: int64

TCH Nucléaire (%)      0
TCH Thermique (%)      0
TCH Eolien (%)         0
TCH Hydraulique (%)    0
TCH Solaire (%)        0
TCH Bioénergies (%)    0
dtype: int64

TCH Nucléaire (%)      0
TCH Thermique (%)      0
TCH Eolien (%)         0
TCH Hydraulique (%)    0
TCH Solaire (%)        0
TCH Bioénergies (%)    0
TCO Nucléaire (%)      0
TCO Thermique (%)      0
TCO Eolien (%)         0
TCO Hydraulique (%)    0
TCO Solaire (%)        0
TCO Bioénergies (%)    0
dtype: int64

## Valeurs Négatives

In [ ]:
# Traitement des valeurs négatives dans les colonnes de production

# On considère que les valeurs de production ne peuvent pas être négatives.
col_production = ['Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)', 'Bioénergies (MW)']

# 1 : Vérifier la présence de valeurs négatives
display((df_clean[col_production] < 0).sum())

# 2 : Remplacer les valeurs négatives par 0
for col in col_production:
    df_clean[col] = df_clean[col].apply(lambda x: 0 if x < 0 else x)

# Vérifier que toutes les valeurs < 0 ont bien été corrigées
display((df_clean[col_production] < 0).sum())

# Interprétation :
# Certaines valeurs de production étaient négatives. C'était probablement des erreurs de saisie ou de capteurs.
# Elles ont été remplacées par 0 car une production ne peut pas être inférieure à zéro.

Thermique (MW)      78327
Nucléaire (MW)        452
Eolien (MW)           619
Solaire (MW)        33437
Hydraulique (MW)        8
Bioénergies (MW)        0
dtype: int64

Thermique (MW)      0
Nucléaire (MW)      0
Eolien (MW)         0
Solaire (MW)        0
Hydraulique (MW)    0
Bioénergies (MW)    0
dtype: int64

# Statistiques descriptives

In [ ]:
# Statistiques descriptives

# On affiche les statistiques descriptives pour chaque variable numérique du dataset nettoyé
# Cela permet d'étudier la distribution des données : moyenne, min, max, etc.
df_clean.describe().T

# Interprétation :
# Les valeurs de production (ex : Thermique, Nucléaire…) présentent des moyennes positives cohérentes.
# Les colonnes TCH (%) ont bien été corrigées et ont un maximum à 100 % comme prévu.
# Les colonnes TCO (%) peuvent dépasser 100 % : cela indique une production excédentaire par rapport à la consommation (TCO Nucléaire, Eolien et Hydrolique).
# Les colonnes 'Pompage (MW)' et 'Ech. physiques (MW)' ont une moyenne négative, ce qui est logique car ce sont des consommations ou des flux d'énergie sortants.
# La colonne 'JourSemaine' varie de 0 à 6 comme attendu (0 = lundi, 6 = dimanche).

,count,mean,min,25%,50%,75%,max,std
Date,2121396,2018-01-16 00:14:59.760724992,2013-01-01 00:00:00,2015-07-10 00:00:00,2018-01-16 00:00:00,2020-07-25 00:00:00,2023-01-31 00:00:00,NaN
Consommation (MW),2121396.0000,4473.4281,703.0000,2768.0000,4102.0000,5708.0000,15338.0000,2162.1170
Thermique (MW),2121396.0000,382.0590,0.0000,25.0000,161.0000,501.0000,4293.0000,540.5401
Nucléaire (MW),2121396.0000,5894.8440,0.0000,3304.0000,6087.0000,8577.5799,13632.0000,3357.7281
Eolien (MW),2121396.0000,265.7032,0.0000,29.0000,109.0000,309.0000,4971.0000,450.9016
Solaire (MW),2121396.0000,96.9630,0.0000,0.0000,0.0000,70.0000,2882.0000,238.5324
Hydraulique (MW),2121396.0000,589.4726,0.0000,5.0000,55.0000,861.0000,8468.0000,1014.2686
Pompage (MW),2121396.0000,-82.0162,-2580.0000,-13.7172,-0.7143,0.0000,0.0000,249.3818
Bioénergies (MW),2121396.0000,84.9507,4.0000,49.0000,77.0000,117.0000,2300.0000,45.2492
Ech. physiques (MW),2121396.0000,-439.0067,-12787.0000,-4044.0000,579.0000,2214.0000,13959.0000,4297.2977


In [ ]:
# Le jeu de données est désormais nettoyé :

# Colonnes inutiles supprimées :
# Column 30 : colonne vide ajoutée par erreur,
# Stockage batterie, Déstockage batterie, Eolien terrestre, Eolien offshore : colonnes entièrement composées de NaN ou de 0, donc non exploitables

# Types de données corrigés : dates converties au bon format

# Doublons supprimés.

# Valeurs manquantes traitées :
# par interpolation temporelle quand c'était possible : quand il y avait des valeurs numériques continues,
# ou par remplissage avec 0 pour les colonnes vides avant 2020.

# Valeurs aberrantes corrigées :
# Pourcentages TCH ramenés à 100 % quand ils dépassaient,
# Valeurs de production négatives remplacées par 0.

# Le dataset est maintenant propre et prêt pour les analyses et visualisations.

# Ajout de colonnes pertinentes pour l'analyse

In [ ]:
# Fonction permettant la création d’un dictionnaire pour l’agrégation par jour :
def GetDico(df):
    dico = {}
    for i in df.columns.values:
        if is_numeric_dtype(df[f'{i}']) and (i != 'Code INSEE région') and (i != 'JourSemaine') :
            if 'TCO' in i or 'TCH' in i:
                dico[i] = 'mean'  # moyenne pour les pourcentages
            else:
                dico[i] = 'sum'   # somme pour les autres valeurs numériques
    return dico

# Création de colonnes personnalisées : ajout de colonnes PROD et CONSO totale

# 1 : Ajout de la production totale 'PROD'
df_clean['PROD'] = (
    df_clean['Thermique (MW)'] +
    df_clean['Nucléaire (MW)'] +
    df_clean['Eolien (MW)'] +
    df_clean['Solaire (MW)'] +
    df_clean['Hydraulique (MW)'] +
    df_clean['Bioénergies (MW)']
)

# 2 : Création de la colonne 'CONSO' (consommation totale du réseau) :
# on additionne la Consommation (MW) (celle des ménages, entreprises…) et le Pompage (MW).
# Explication : le pompage est une forme de consommation.
# Quand on a trop d’électricité (par exemple : la nuit), on pompe de l’eau vers un réservoir en altitude. Cela consomme de l’électricité à ce moment-là.
# Plus tard, on laisse redescendre cette eau pour produire à nouveau de l’électricité par production hydraulique.
df_clean['CONSO'] = df_clean['Consommation (MW)'] + df_clean['Pompage (MW)']

# 3 : Agrégation par région / date
df_clean = df_clean.groupby(['Code INSEE région', 'Région', 'Date', 'JourSemaine']).agg(GetDico(df_clean))
df_clean = df_clean.reset_index()

# 4 : Ajout de colonne Année-mois, mois et année
df_clean['Année_mois'] = df_clean['Date'].astype(str).str[:7]
df_clean['Année'] = df_clean['Date'].astype(str).str[:4]
df_clean['Mois'] = df_clean['Date'].astype(str).str[5:7]

# 5. Correction du Pompage : le pompage est en valeurs négatives, on le remet en valeur absolue
df_clean['Pompage (MW)'] = abs(df_clean['Pompage (MW)'])

# En résumé :
# On ajoute la production totale (PROD) et la consommation nette (CONSO)
# On extrait l’année, le mois et l’année-mois pour faciliter les analyses temporelles
# On regroupe les données par date et région : Somme pour toutes les puissances (MW) et Moyenne pour les pourcentages (TCO, TCH)
# On corrige à nouveau la colonne Pompage si des valeurs sont redevenues négatives

In [ ]:
df_clean.head(20)

,Code INSEE région,Région,Date,JourSemaine,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),PROD,CONSO,Année_mois,Année,Mois
0,11,Île-de-France,2013-01-01,1,399392.0000,32478.0000,508590.9718,340.0000,100.0000,0.0000,117.5000,6333.0000,360051.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,547841.9718,399274.5000,2013-01,2013,01
1,11,Île-de-France,2013-01-02,2,492157.0000,33589.0000,532984.0552,83.0000,121.0000,0.0000,96.0000,6016.0000,452266.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,572793.0552,492061.0000,2013-01,2013,01
2,11,Île-de-France,2013-01-03,3,487111.0000,42885.0000,531158.2345,120.0000,16.0000,0.0000,1152.0000,6136.0000,437869.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,580315.2345,485959.0000,2013-01,2013,01
3,11,Île-de-France,2013-01-04,4,470053.0000,34260.0000,527388.4138,12.0000,15.0000,0.0000,15600.0000,6273.0000,429408.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,567948.4138,454453.0000,2013-01,2013,01
4,11,Île-de-France,2013-01-05,5,433732.0000,33634.0000,521453.9586,4.0000,13.0000,0.0000,96.0000,6585.0000,393424.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,561689.9586,433636.0000,2013-01,2013,01
5,11,Île-de-France,2013-01-06,6,430898.0000,33660.0000,536821.4069,0.0000,17.0000,0.0000,96.0000,6577.0000,390575.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,577075.4069,430802.0000,2013-01,2013,01
6,11,Île-de-France,2013-01-07,0,496093.0000,47514.0000,551499.4759,9.0000,37.0000,0.0000,1968.0000,6207.0000,442252.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,605266.4759,494125.0000,2013-01,2013,01
7,11,Île-de-France,2013-01-08,1,525381.0000,52653.0000,559280.1103,0.0000,9.0000,0.0000,1968.0000,6264.0000,466386.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,618206.1103,523413.0000,2013-01,2013,01
8,11,Île-de-France,2013-01-09,2,540858.0000,51245.0000,558394.2621,24.0000,24.0000,0.0000,120.0000,6169.0000,483302.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,615856.2621,540738.0000,2013-01,2013,01
9,11,Île-de-France,2013-01-10,3,523950.0000,52081.0000,551577.2690,65.0000,23.0000,0.0000,7848.0000,5940.0000,465730.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,609686.2690,516102.0000,2013-01,2013,01


In [ ]:
# Conclusion : le jeu de données est désormais nettoyé :
# colonnes vides ou inutiles supprimées
# types temporels convertis correctement (Date, Heure, Date-Heure)
# valeurs manquantes traitées (interpolation ou remplissage)
# doublons supprimés
# valeurs aberrantes corrigées (valeurs négatives ou pourcentages incohérents)
# colonnes personnalisées ajoutées pour faciliter l’analyse (PROD, CONSO, Année, Mois)
# Le DataFrame `df_clean` est prêt pour une analyse temporelle et régionale fiable.

In [ ]:
# pour enregistrer et télécharger le fichier en local, pour garder une trace de la progression.
df_clean.to_csv("eco2mix_clean.csv", index=False)

In [ ]:
# FACULTATIF
#from google.colab import files
#files.download("eco2mix_clean.csv")